# Algoritmo A*

Precisamos de importar a classe ConnectFour do ficheiro connect4.py

In [ ]:
from connect4 import ConnectFour

Criamos a classe Astar:

De seguida criamos a função __evaluate__ que recebe um segmento do tabuleiro (linha, coluna ou diagonal) e retorna duas pontuações: uma para o jogador e outra para o seu oponente (neste caso, o algoritmo A*)

Inicialmente a função verifica se o segmento tem um comprimento de pelo menos 4 células e se tem 3 X's consecutivos. Se isto se verificar incrementa a pontuação em 100 pontos. Isto foi uma forma de tentar otimizar o algoritmo dando prioridade a situações em que o jogador X está prestes a ganhar.

__player_count__, __opponent_count__ e __empty_count__: contam o número de peças do jogador atual, do oponente e de espaços vazios no segmento do tabuleiro fornecido.

Tendo em conta os contadores, são atribuidas pontuações de acordo com as seguintes condições:
- Se tiver 4 peças em linha, adiciona 512
- Se tiver 3 peças em linha e 1 célula vazia, adiciona 50
- Se tiver 2 peças em linha e 2 células vazias, adiciona 10
- Se tiver 1 peça e 3 células vazias, adiciona 1
- Caso contrário não altera a pontuação

Para o oponente, segue a mesma lógica mas vai subtraindo em vez de somar.

In [ ]:
class Astar:
    
    def evaluate(segment, player):
        score = 0

        if len(segment) >= 4 and Astar.check_three_xs(segment, ConnectFour().PLAYER_X):
            score += 100

        player_count = 0
        opponent_count = 0
        empty_count = 0

        for token in segment:
            if token == ConnectFour().PLAYER_X:
                player_count += 1
            elif token == ConnectFour().PLAYER_O:
                opponent_count += 1
            elif token == ConnectFour().EMPTY:
                empty_count += 1

        if player_count == 4:
            score += 512
        elif player_count == 3 and empty_count == 1:
            score += 50
        elif player_count == 2 and empty_count == 2:
            score += 10
        elif player_count == 1 and empty_count == 3:
            score += 1
        else:
            score += 0

        if opponent_count == 4:
            score -= 512
        elif opponent_count == 3 and empty_count == 1:
            score -= 50
        elif opponent_count == 2 and empty_count == 2:
            score -= 10
        elif opponent_count == 1 and empty_count == 3:
            score -= 1
        else:
            score -= 0

        return score

Criamos também uma função __heuristic__, que calcula a heurística para uma dada configuração do tabuleiro. A heurística consiste numa avaliação do quão favorável é uma posição para um jogador, sem examinar todas as jogadas futuras possíveis.

Voltamos a verificar a existência de 3 X's consecutivos como forma de garantir que ambos os métodos estão alinhados e usam a mesma lógica, tornando o algoritmo mais eficiente.

Depois chama-se a função __evaluate__ de forma a se avaliar os segmentos horizontais, verticais e diagonais(crescentes e decrescentes) do tabuleiro, respetivamente.
Vamos subtraindo 3 ao nº de linhas e colunas, pois são precisas 4 peças consecutivas para ganhar e assim evita-se passar os limites do tabuleiro.

In [ ]:
    def heuristic(board):
        score = 0

        if Astar.check_three_xs(board, ConnectFour().PLAYER_X):
            score += 100

        for row in range(ConnectFour().ROWS):
            for col in range(ConnectFour().COLUMNS - 3):
                segment = board[row][col:col+4]
                score += Astar.evaluate(segment, ConnectFour().PLAYER_X)

        for row in range(ConnectFour().ROWS - 3):
            for col in range(ConnectFour().COLUMNS):
                segment = [board[row+i][col] for i in range(4)]
                score += Astar.evaluate(segment, ConnectFour().PLAYER_X)

        for row in range(ConnectFour().ROWS - 3):
            for col in range(ConnectFour().COLUMNS - 3):
                segment = [board[row+i][col+i] for i in range(4)]
                score += Astar.evaluate(segment, ConnectFour().PLAYER_X)

        for row in range(ConnectFour().ROWS - 3):
            for col in range(ConnectFour().COLUMNS - 3, -1, -1):
                segment = [board[row+i][col-i] for i in range(4)]
                score += Astar.evaluate(segment, ConnectFour().PLAYER_X)

        return score

Seguidamente, precisamos de criar uma classe __Node__ que é utilizada para representar nós, que são necessários para explorar o espaço de busca de forma eficiente, selecionando os nós que devem ser explorados a seguir.

A função __init__ é o construtor da classe que contém os seguintes atributos:
- board -> que representa o estado do tabuleiro associado a este nó
- parent -> que representa o nó pai deste nó no espaço de busca
- move -> que representa a ação que levou o estado do tabuleiro atual a partir do estado do tabuleiro do nó pai
- g -> que representa o custo acumulado para chegar a este nó a partir do nó inicial
- h -> que representa a heurística estimada para chegar ao objetivo a partir deste nó
- f -> que representa a soma dos custos (g + h)

O método def __eq__(self, other) compara os tabuleiros dos nós para determinar se são iguais e o método def __hash__(self) gera um hash baseado na representação do tabuleiro como uma string.

In [ ]:
    class Node:
        def __init__(self, board, parent, move, g=0, h=0, f=0):
            self.board = board
            self.parent = parent
            self.move = move
            self.g = g
            self.h = h
            self.f = f

        def __eq__(self, other):
            return self.board == other.board


        def __hash__(self):
            return hash(str(self.board))

Depois, criamos a função __astar__ que implementa o algoritmo A* e retorna o nó correspondente ao melhor caminho encontrado.

Primeiro criamos um nó inicial com o tabuleiro fornecido, que não tem um nó pai uma vez que é o nó inicial e nenhuma ação pois é o inicio da pesquisa.
De seguida, criamos um conjunto __open_nodes__ que inicialmente contém o nó inicial, e onde vão ser adicionados os nós a serem explorados pelo algoritmo.
O conjunto __closed_nodes__, é um conjunto vazio que vai armazenar os nós que já foram explorados.

Enquanto houverem nós para ser explorados, a variável __lowest_f__ vai procurar o nó com o menor valor de f. À medida que isto vai acontecendo os nós vão sendo removidos do conjunto __open_nodes__ e vão sendo adicionados ao __closed_nodes__, pois já foram explorados.

Posteriormente, verifica-se se o estado atual do tabuleiro contém uma posição vencedora. Se sim, retorna-se o nó atual, que representa o melhor caminho encontrado até agora.

Usando a função __create_child__, verifica-se se algum nó filho é igual a algum nó já explorado e se isto acontecer, esse nó filho é adicionado ao conjunto dos nós já explorados.

In [ ]:
    def astar(board, player):
        initial = Astar.Node(board, None, None)
        open_nodes = {initial}
        closed_nodes = set()

        while open_nodes:

            lowest_f = float('inf')
            for node in open_nodes:
                if node.f < lowest_f:
                    lowest_f = node.f
                    current = node

            open_nodes.remove(current)
            closed_nodes.add(current)

            if ConnectFour().win(current.board, ConnectFour().PLAYER_O if player else ConnectFour().PLAYER_X):
                return current

            for child in Astar.create_child(current, player):
                explored = False

                for node in closed_nodes:
                    if child == node:
                        explored = True
                        break

                if not explored:
                    open_nodes.add(child)

        return None

A função __create_child__ é responsável por gerar os nós filhos a partir de um nó pai.

Primeiro, inicializamos uma lista vazia para armazenar os nós filhos que vão ser gerados (__children__) e outra para armazenar os movimentos que podem ser feitos a partir do estado em que está o tabuleiro (__valid_moves__).

Percorre-se todas as colunas do tabuleiro e utilizando a função __is_valid_column__ da classe ConnectFour, vão-se adicionando á lista valid_moves os movimentos que são possíveis. Com isto e usando a função __empty_row__ da classe ConnectFour determina-se a linha onde a peça deve ser inserida.
Cria-se uma cópia do tabuleiro do nó pai e a peça é inserida na posição determinada anteriormente.

Depois, verifica-se o seguinte:
- se for a vez do jogador atual e ele tem a oportunidade de ganhar ao inserir uma peça na posição seguinte então é gerado um nó filho que representa essa possível vitória.
- se for a vez do oponente e não for permitido que o oponente ganhe na próxima jogada, então não é gerado um nó filho.

Caso contrário, um nó filho é gerado para representar o estado do tabuleiro após o movimento.

Os custos g,h e f do nó filho são atualizados com base no nó pai e na heurística do novo estado do tabuleiro.

Por fim, o nó filho é adicionado á lista de nós filhos.

In [ ]:
    def create_child(node, player):
        children = []
        valid_moves = []

        for col in range(ConnectFour().COLUMNS):
            if ConnectFour().is_valid_column(node.board, col):
                valid_moves.append(col)

        for col in valid_moves:
            row = ConnectFour().empty_row(node.board, col)
            new_board = ConnectFour().copy_board(node.board)
            ConnectFour().drop_token(new_board, row, col, ConnectFour().PLAYER_O if player else ConnectFour().PLAYER_X)

            if player and Astar.check_three_xs(new_board, ConnectFour().PLAYER_O):
                next_row = ConnectFour().empty_row(new_board, col)
                next_new_board = ConnectFour().copy_board(new_board)
                if next_row < ConnectFour().ROWS - 1:
                    ConnectFour().drop_token(next_new_board, next_row + 1, col, ConnectFour().PLAYER_O)
                    if not Astar.check_three_xs(next_new_board, ConnectFour().PLAYER_O):
                        new_node = Astar.Node(next_new_board, node, col)
                        new_node.g = node.g + 1
                        new_node.h = Astar.heuristic(next_new_board)
                        new_node.f = new_node.g + new_node.h
                        children.append(new_node)

            elif not player and Astar.check_three_xs(new_board, ConnectFour().PLAYER_X):
                continue

            else:
                new_node = Astar.Node(new_board, node, col)
                new_node.g = node.g + 1
                new_node.h = Astar.heuristic(new_board)
                new_node.f = new_node.g + new_node.h
                children.append(new_node)

        return children

Por fim, criamos a função __check_three_xs__ que verifica se há três peças consecutivas do mesmo jogador, como forma de otimizar o algoritmo. Ou seja, quando o jogador_X está perto de ganhar o jogador_Y deve tentar impedi-lo. 

In [ ]:
    def check_three_xs(board, player):

        for row in range(len(board) - 3):
            for col in range(len(board[row]) - 3):
                if board[row][col] == player and board[row+1][col+1] == player and board[row+2][col+2] == player and board[row+3][col+3] == player:
                    return True

        for row in range(len(board)):
            for col in range(len(board[row]) - 3):
                if board[row][col] == player and board[row][col+1] == player and board[row][col+2] == player and board[row][col+3] == player:
                    return True

        return False